In [69]:
import chess
from IPython.display import clear_output
from IPython.display import display
import json
import time
import random

In [70]:
fen = "8/8/8/8/4k3/8/8/K6R w - - 0 1"
board = chess.Board(fen)

In [71]:
S_n_sequence_new = []
f = open("S_n_seq.json", "r")
tmp = json.loads(f.read())
for item in tmp:
    tmp_list = []
    for board in item:
        tmp_list.append(chess.Board(board))
    S_n_sequence_new.append(tmp_list)
f.close()

In [72]:
compare_board = chess.Board(fen)
board_str = (compare_board.turn, compare_board.__str__())
s_index = 0
board_index = 0

for i in range(len(S_n_sequence_new)):
    for j in range(len(S_n_sequence_new[i])):
        item = S_n_sequence_new[i][j]
        if board_str == (item.turn, item.__str__()):
            s_index = i
            board_index = j
            break
    if s_index != 0:
        break

print("Ausgangssituation gefunden in: S" + str(s_index) + " - Board: " + str(j))

Ausgangssituation gefunden in: S9 - Board: 19108


In [73]:
def checkForBoard(repr, list):
    for i in range(len(list)):
            next_board = list[i]
            if repr == (next_board.turn, next_board.__str__()):
                return i

    return 0
moves = []

while s_index > 0:
    curr_board = S_n_sequence_new[s_index][board_index]
    if curr_board.turn:
        move_count = 1
        for move in curr_board.legal_moves:
            print(move_count)
            move_count = move_count + 1
            curr_board.push(move)
            repr = (curr_board.turn, curr_board.__str__())
            next_index = checkForBoard(repr, S_n_sequence_new[s_index - 1])
            if next_index:
                moves.append(move)
                s_index = s_index - 1
                board_index = next_index
                break
            curr_board.pop()
    else:
        move_list = list(curr_board.legal_moves)
        move_index = random.randint(0, len(move_list) - 1)
        curr_board.push(move_list[move_index])
        repr = (curr_board.turn, curr_board.__str__())
        next_index = checkForBoard(repr, S_n_sequence_new[s_index - 1])
        s_index = s_index - 1
        board_index = next_index
        moves.append(move_list[move_index])

    print("---")


clear_output(wait=True)
print(str(len(moves)) + " Züge bis zum Gewinn bestimmt")

1
2
---
---
1
2
---
---
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
---
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
---
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
---
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
---
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
---
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
---
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
---
1
2
3
4
5
6
7
8
9
10
11
12


KeyboardInterrupt: 

In [ ]:
test_board = compare_board.copy()
display(test_board)
time.sleep(2)
for move in moves:
    test_board.push(move)
    clear_output(wait=True)
    display(test_board)
    time.sleep(2)
